In [7]:
import numpy as np
import os
from Bio import SeqIO #pip install biopython
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [8]:
#step 1
def generate_motif(ML, ICPC): 
    if ICPC == 2:
        results = []
        for i in range(ML):
            col = [0,0,0,0]
            loc = np.random.randint(0,4)
            col[loc] = 1
            results.append(col)
        return results
    
    
    eplison = 0.01
    
    def n_sum(ICs, ML, ICPC):
        if ML == 6:
            for a in range(0, len(ICs) - 5):
                for b in range(1, len(ICs) - 4):
                    for c in range(2, len(ICs) - 3):
                        for d in range(3, len(ICs) - 2):
                            for e in range(4, len(ICs) - 1):
                                for f in range(5, len(ICs)):
                                    if abs(ICs[a] + ICs[b] + ICs[c] + ICs[d] + ICs[e] + ICs[f] - ICPC) <  eplison and len(set([a,b,c,d,e,f])) == 6:
                                        return [a,b,c,d,e,f]
        elif ML == 7:
            for a in range(0, len(ICs) - 6):
                for b in range(1, len(ICs) - 5):
                    for c in range(2, len(ICs) - 4):
                        for d in range(3, len(ICs) - 3):
                            for e in range(4, len(ICs) - 2):
                                for f in range(5, len(ICs) - 1):
                                    for g in range(5, len(ICs)):
                                        if abs(ICs[a] + ICs[b] + ICs[c] + ICs[d] + ICs[e] + ICs[f] + ICs[g] - ICPC) <  eplison and len(set([a,b,c,d,e,f,g])) == 7:
                                            return [a,b,c,d,e,f,g]
        else: #8
            for a in range(0, len(ICs) - 7):
                for b in range(1, len(ICs) - 6):
                    for c in range(2, len(ICs) - 5):
                        for d in range(3, len(ICs) - 4):
                            for e in range(4, len(ICs) - 3):
                                for f in range(5, len(ICs) - 2):
                                    for g in range(5, len(ICs) - 1):
                                        for h in range(6, len(ICs)):
                                            if abs(ICs[a] + ICs[b] + ICs[c] + ICs[d] + ICs[e] + ICs[f] + ICs[g] + ICs[h] - ICPC) <  eplison and len(set([a,b,c,d,e,f,g,h])) == 8:
                                                return [a,b,c,d,e,f,g,h]
        return []
    
    def calc_IC(col):
        return sum([col[i]*np.log2(4 * col[i]) for i in range(4)])
    
    
    idxs = []
    while len(idxs) == 0:
        cols = []
        ICs = []
        for i in range(ML*2):
            rvs = sorted(np.random.uniform(size = 3))
            col = [rvs[0] - 0, rvs[1] - rvs[0], rvs[2] - rvs[1], 1 - rvs[2]]
            cols.append(col)
            ICs.append(calc_IC(col))
        idxs = n_sum(ICs, ML, ICPC)
    
    results = []
    for idx in idxs:
        results.append(cols[idx])
        
    return results
    
    
def build_benchmark(ICPC, ML, SL, SC, index):
    ACGT = ['A', 'C', 'G', 'T']
    #2)
    sequences = []
    for i in range(SC):
        sequence = []
        for j in range(SL):
            k = np.random.randint(0,4)
            sequence.append(ACGT[k])
        sequences.append(sequence)
    #3)
    motif = generate_motif(ML, ICPC)
    #4)
    sites = []
    for i in range(SC):
        site = []
        for j in range(ML):
            k = np.random.uniform()
            if k < motif[j][0]:
                site.append(ACGT[0])
            elif k >= motif[j][0] and k < motif[j][0] + motif[j][1]:
                site.append(ACGT[1])
            elif k >= motif[j][0] + motif[j][1] and k < motif[j][0] + motif[j][1] + motif[j][2]:
                site.append(ACGT[2])
            else:
                site.append(ACGT[3])
        sites.append(site)
    #5)
    sites_location = []
    for i in range(SC):
        j = np.random.randint(0,SC) #check idx
        k = np.random.randint(0,SL - ML) #check idx
        sites[i][k: k + ML] = sites[j] #check idx
        sites_location.append(k)
    #6)
    location = 'data_set_' + str(index)
    if not os.path.exists(location):
        os.makedirs(location)
    
    s = []   
    for i in sequences:
        s.append(SeqRecord(Seq(''.join(i))))
    
    SeqIO.write(s, os.path.join(location, 'sequences' + ".fasta"), "fasta")
    
    #7)
    f = open(os.path.join(location, 'sites' + ".txt"), 'w+')
    for i in sites_location:
        f.write(str(i))
        f.write(',')
    f.close()
    #8)
    f = open(os.path.join(location, 'motif' + ".txt"), 'w+')
    f.write('MOTIF' + str(index) + ' ' + str(ML))
    f.write('\n')
    for i in motif:
        for j in i:
            f.write(str(j))
            f.write(',')
        f.write('\n')
    f.close()
    #9)
    with open(os.path.join(location, 'motiflength' + ".txt"), 'w+') as f:
        f.write(str(ML))
        #f.close()

In [9]:
for i in range(10):
    build_benchmark(1,7,500,10,0 + i)
for i in range(10):
    build_benchmark(1.5,7,500,10,10 + i)
for i in range(10):
    build_benchmark(2,7,500,10,20 + i)
for i in range(10):
    build_benchmark(1.5,6,500,10,30 + i)
for i in range(10):
    build_benchmark(1.5,8,500,10,40 + i)
for i in range(10):
    build_benchmark(1.5,7,500,5,50 + i)
for i in range(10):
    build_benchmark(1.5,7,500,20,60 + i)

In [4]:
#step 2
def motif_finder(index):
    location = 'data_set_' + str(index) + '/' 
    for record in SeqIO.parse(os.path.join(location, 'sequences' + ".fasta"), "fasta"):
        #print(record)
        pass
    with open(os.path.join(location,'motiflength.txt'), 'r') as f:
        ML = int(f.read())
        #f.close()    
    
    motif = []
    sites = []
    

    location = 'data_set_' + str(index)
    f = open(os.path.join(location, 'predicted_motif' + ".txt"), 'w+')
    f.write('MOTIF' + str(index) + ' ' + str(ML))
    f.write('\n')
    for i in motif:
        for j in i:
            f.write(str(j))
            f.write(',')
        f.write('\n')
    f.close()
    
    f = open(os.path.join(location, 'predicted_sites' + ".txt"), 'w+')
    for i in sites:
        f.write(str(i))
        f.write(',')
    f.close()


In [5]:
#step 3
def evaluation(index):
    location = '/data_set_' + str(index) + '/' 
    with open(os.path.join(location,'motif.txt'), 'r') as f:
        motif = []
    with open(os.path.join(location,'predicted_motif.txt'), 'r') as f:
        predicted_motif = []
    with open(os.path.join(location,'sites.txt'), 'r') as f:
        sites = f.readlines()[0].split(',')[:-1]
    with open(os.path.join(location,'predicted_sites.txt'), 'r') as f:
        predicted_sites = f.readlines()[0].split(',')[:-1]
    #1)
    def KL(a, b):
        a = np.asarray(a, dtype=np.float)
        b = np.asarray(b, dtype=np.float)
        return np.sum(np.where(a != 0, a * np.log(a / b), 0))

    a = sum([KL(motif[i], predicted_motif[i]) for i in range(len(motif))])
    #2)
    b = sum([1 for i in range(len(sites)) if sites[i] == predicted_sites[i]])
    #3)
    c = sum([1 for i in range(len(sites)) if abs(sites[i] - predicted_sites[i]) < len(motif)/2])
    #4)
    d = 0
    
    return a, b, c, d

In [6]:
motif_finder(0)